# Demonstration on loading files into parquet

#### Goal

- To load a sample of a raw text file and transform into parquet file
- Examine metadata of parquet partitions
- Load parquet into a pandas dataframe

#### Load libraries

In [1]:
! pip install pyarrow

In [2]:
# General tools & operations libraries
import re
import ast
import time
import csv
import itertools

# Mathematical operations and dataframes libraries
import numpy as np
import pandas as pd

# Plotting and visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Parquet libraries
import pyarrow as pa
import pyarrow.parquet as pq

# PySpark libraries
from pyspark.sql import SQLContext
#from pyspark.sql import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.conf import SparkConf

#### Set parameters and Spark configurations

In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
# store path to notebook
PWD = !pwd
PWD = PWD[0]

In [5]:
# assign parameters
!BUCKET=danielalvarez_w261projects

In [6]:
# Set input and outfiles to store parquet files
INPUT_FILES = 'data/sample.txt'
#INPUT_FILES = 'gs://danielalvarez_w261projects/finalproject/sample.txt'
OUT_FILES = 'gs://danielalvarez_w261projects/finalproject/df.parquet'

In [7]:
# start Spark Session
from pyspark.sql import SparkSession
app_name = "finalproject_notebook"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [8]:
# Spark configuration Information
for object in sc.getConf().getAll():
    print(object)

('spark.rdd.compress', 'True')
('spark.driver.port', '37043')
('spark.serializer.objectStreamReset', '100')
('spark.master', 'local[*]')
('spark.executor.id', 'driver')
('spark.submit.deployMode', 'client')
('spark.app.name', 'finalproject_notebook')
('spark.app.id', 'local-1575325563237')
('spark.ui.showConsoleProgress', 'true')
('spark.driver.host', 'docker.w261')


In [9]:
spark

#### Read in text file and create dataframe

In [10]:
# check number of bytes, lines, and words
!wc data/sample.txt

  183029  6305520 44503676 data/sample.txt


In [11]:
# inspect first line
!head -n 1 data/sample.txt

0	2	-1			501	0	2	0	0	1	1			68fd1e64	4c2bc594	d032c263	c18be181	25c83c98	fe6b92e5	1e9876db	0b153874	a73ee510	fa7d0797	043725ae	dfbb09fb	7f0d7407	8ceecbc8	7ac43a46	84898b2a	07c540c4	bc48b783			0014c32a		3a171ecb	3b183c5c		


Read the file in as a Spark dataframe and check the schema

In [12]:
# check the schema 
df = sqlContext.read.format("com.databricks.spark.csv").option("header", "true").load("data/sample.txt")

In [13]:
df.printSchema()

root
 |-- 0	2	-1			501	0	2	0	0	1	1			68fd1e64	4c2bc594	d032c263	c18be181	25c83c98	fe6b92e5	1e9876db	0b153874	a73ee510	fa7d0797	043725ae	dfbb09fb	7f0d7407	8ceecbc8	7ac43a46	84898b2a	07c540c4	bc48b783			0014c32a		3a171ecb	3b183c5c		: string (nullable = true)



In [14]:
# Count number of rows
df.count()

183028

Impose schema structure. 

The 13th variable is a numeric (`n13`), 14th variable is categorical (`cat14`)

In [15]:
# the 13th variable is a numeric (`n13`), 14th variable is categorical (`cat14`)
schema = StructType([
    StructField('y', IntegerType()),
    StructField('n1', IntegerType()),
    StructField('n2', IntegerType()),
    StructField('n3', IntegerType()),
    StructField('n4', IntegerType()),
    StructField('n5', LongType()),
    StructField('n6', IntegerType()),
    StructField('n7', IntegerType()),
    StructField('n8', IntegerType()),
    StructField('n9', IntegerType()),
    StructField('n10', IntegerType()),
    StructField('n11', IntegerType()),
    StructField('n12', IntegerType()),
    StructField('n13', IntegerType()), 
    StructField('cat14', StringType()),
    StructField('cat15', StringType()),
    StructField('cat16', StringType()),
    StructField('cat17', StringType()),
    StructField('cat18', StringType()),
    StructField('cat19', StringType()),
    StructField('cat20', StringType()),
    StructField('cat21', StringType()),
    StructField('cat22', StringType()),
    StructField('cat23', StringType()),
    StructField('cat24', StringType()),
    StructField('cat25', StringType()),
    StructField('cat26', StringType()),
    StructField('cat27', StringType()),
    StructField('cat28', StringType()),
    StructField('cat29', StringType()),
    StructField('cat30', StringType()),
    StructField('cat31', StringType()),
    StructField('cat32', StringType()),
    StructField('cat33', StringType()),
    StructField('cat34', StringType()),
    StructField('cat35', StringType()),
    StructField('cat36', StringType()),
    StructField('cat37', StringType()),
    StructField('cat38', StringType()),
    StructField('cat39', StringType()),
])

Create Spark dataframe

In [16]:
start = time.time()
print('Creating dataframe..')
df = spark.read.load("data/sample.txt", format='csv', sep='\t', header='false', schema=schema)
#df = spark.read.load(INPUT_FILES, format='csv', sep='\t', header='false', schema=schema)
print(f"... completed job in {time.time() - start} seconds")

Creating dataframe..
... completed job in 0.09658217430114746 seconds


Show the first 5 rows of selected columns

In [17]:
print(df.select('y','n1','n12','n13','cat14','cat39').show(n=5))

+---+----+----+----+--------+--------+
|  y|  n1| n12| n13|   cat14|   cat39|
+---+----+----+----+--------+--------+
|  0|   2|null|null|68fd1e64|    null|
|  0|   3|   0|   1|68a25dc5|da9fe092|
|  0|   0|null|   2|68fd1e64|    null|
|  0|  64|   0|null|05db9164|    null|
|  1|null|null|null|8cf07265|0a47000d|
+---+----+----+----+--------+--------+
only showing top 5 rows

None


In [18]:
# Count the number of rows
df.count()

183029

### Write dataframe to parquet file format

In [19]:
start = time.time()
print('Writing dataframe to parquet format..')

df.write.parquet('data/df.parquet', compression='snappy', mode='overwrite')
#df.write.parquet(OUT_FILES, compression='snappy', mode='overwrite')

print(f"... completed job in {time.time() - start} seconds")

Writing dataframe to parquet format..
... completed job in 4.0193445682525635 seconds


### Read in parquet files

In [20]:
df_pq = spark.read.load('data/df.parquet')

In [21]:
# count the number of rows
print(df_pq.count())

# perform an assert to check number of rows matches before and after parquet conversion
print(df_pq.count() == df.count())

183029
True


The gsutil ls command with options -l (long listing) and -R (recursive listing) will list the entire bucket recursively and then produce a total count of all objects, both files and directories, at the end

In [22]:
# ! gsutil ls -lR gs://w261-f19-team1/train

Get the total, piping in the `tail` command

In [23]:
# ! gsutil ls -lR gs://w261-f19-team1/train | tail -n 1

Count the number of files, bytes, and words

In [24]:
# ! gsutil ls -lR gs://w261-f19-team1/train | wc 

#### Load parquet partition files

In [25]:
!ls -lR data/df.parquet

data/df.parquet:
total 12744
-rw-r--r-- 1 root root       0 Dec  2 22:26 _SUCCESS
-rw-r--r-- 1 root root 3529804 Dec  2 22:26 part-00000-b9c2b470-4ae1-423b-9aca-6976680fdb09-c000.snappy.parquet
-rw-r--r-- 1 root root 3536883 Dec  2 22:26 part-00001-b9c2b470-4ae1-423b-9aca-6976680fdb09-c000.snappy.parquet
-rw-r--r-- 1 root root 3544833 Dec  2 22:26 part-00002-b9c2b470-4ae1-423b-9aca-6976680fdb09-c000.snappy.parquet
-rw-r--r-- 1 root root 2430975 Dec  2 22:26 part-00003-b9c2b470-4ae1-423b-9aca-6976680fdb09-c000.snappy.parquet


Get the total of the last partition, piping in the `tail` command

In [26]:
!ls -lR data/df.parquet | tail -n 1

-rw-r--r-- 1 root root 2430975 Dec  2 22:26 part-00003-b9c2b470-4ae1-423b-9aca-6976680fdb09-c000.snappy.parquet


Count the number of files, bytes, and words

In [27]:
!ls -lR data/df.parquet | wc

      7      48     530


Check the metadata of each partition

In [30]:
partitions = ['data/df.parquet/part-00000-b9c2b470-4ae1-423b-9aca-6976680fdb09-c000.snappy.parquet', 
             'data/df.parquet/part-00001-b9c2b470-4ae1-423b-9aca-6976680fdb09-c000.snappy.parquet',
             'data/df.parquet/part-00002-b9c2b470-4ae1-423b-9aca-6976680fdb09-c000.snappy.parquet',
             'data/df.parquet/part-00003-b9c2b470-4ae1-423b-9aca-6976680fdb09-c000.snappy.parquet']

In [31]:
for file in partitions:
    pqfile = pq.ParquetFile(file)
    #table = pq.read_table('gs://w261-f19-team1/train/part-00003-7fa47152-b757-4ddd-ab8e-14738b4015f0-c000.snappy.parquet')
    print(pqfile.metadata)

  created_by: parquet-mr version 1.10.1 (build a89df8f9932b6ef6633d06069e50c9b7970bebd1)
  num_columns: 40
  num_rows: 50037
  num_row_groups: 1
  format_version: 1.0
  serialized_size: 6608
  created_by: parquet-mr version 1.10.1 (build a89df8f9932b6ef6633d06069e50c9b7970bebd1)
  num_columns: 40
  num_rows: 50139
  num_row_groups: 1
  format_version: 1.0
  serialized_size: 6608
  created_by: parquet-mr version 1.10.1 (build a89df8f9932b6ef6633d06069e50c9b7970bebd1)
  num_columns: 40
  num_rows: 50059
  num_row_groups: 1
  format_version: 1.0
  serialized_size: 6608
  created_by: parquet-mr version 1.10.1 (build a89df8f9932b6ef6633d06069e50c9b7970bebd1)
  num_columns: 40
  num_rows: 32794
  num_row_groups: 1
  format_version: 1.0
  serialized_size: 6593


In [32]:
!wc data/df.parquet/part-00000-6dadf449-f385-4950-8167-6024f329f107-c000.snappy.parquet

wc: data/df.parquet/part-00000-6dadf449-f385-4950-8167-6024f329f107-c000.snappy.parquet: No such file or directory


#### Read parquet file as a pandas dataframe

In [34]:
part1_df = pd.read_parquet('data/df.parquet/part-00000-b9c2b470-4ae1-423b-9aca-6976680fdb09-c000.snappy.parquet', engine='pyarrow')

In [35]:
part1_df.head()

,y,n1,n2,n3,n4,n5,n6,n7,n8,n9,...,cat30,cat31,cat32,cat33,cat34,cat35,cat36,cat37,cat38,cat39
0,0,2.0,-1,NaN,NaN,501.0,0.0,2.0,0.0,0.0,...,07c540c4,bc48b783,None,None,0014c32a,None,3a171ecb,3b183c5c,None,None
1,0,3.0,-1,2.0,1.0,79.0,1.0,3.0,1.0,1.0,...,07c540c4,f54016b9,21ddcdc9,b1252a9d,5b4aa781,None,32c7478e,1793a828,e8b83407,da9fe092
2,0,0.0,0,3.0,2.0,15864.0,283.0,1.0,2.0,155.0,...,27c07bd6,38748bc3,None,None,5a8fe828,None,32c7478e,f96a556f,None,None
3,0,64.0,8,NaN,NaN,177.0,0.0,65.0,20.0,18.0,...,e5ba7672,e8623312,None,None,e106ec2a,None,423fab69,b889075b,None,None
4,1,NaN,-1,NaN,NaN,8020.0,26.0,6.0,0.0,80.0,...,e5ba7672,7119e567,1d04f4a4,b1252a9d,d5f54153,None,32c7478e,a9d771cd,c9f3bea7,0a47000d


In [36]:
part1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50037 entries, 0 to 50036
Data columns (total 40 columns):
y        50037 non-null int32
n1       27891 non-null float64
n2       50037 non-null int32
n3       39260 non-null float64
n4       39178 non-null float64
n5       48694 non-null float64
n6       38527 non-null float64
n7       47884 non-null float64
n8       50012 non-null float64
n9       47884 non-null float64
n10      27891 non-null float64
n11      47884 non-null float64
n12      11489 non-null float64
n13      39178 non-null float64
cat14    50037 non-null object
cat15    50037 non-null object
cat16    48314 non-null object
cat17    48314 non-null object
cat18    50037 non-null object
cat19    43530 non-null object
cat20    50037 non-null object
cat21    50037 non-null object
cat22    50037 non-null object
cat23    50037 non-null object
cat24    50037 non-null object
cat25    48314 non-null object
cat26    50037 non-null object
cat27    50037 non-null object
cat28    5003